In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import random
import math
import helper
import importlib
import h5py
#from netplot import ModelPlot # this fucking library uses a weird version of numpy so we are not gonna use it
from tensorflow.keras.utils import plot_model
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import backend as keras_backend
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SpatialDropout2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.regularizers import l2

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# set the random seed
tf.random.set_seed(42)
np.random.seed(42)
#MFCC_RESNET32_lr-1e-06_b1-0.99_b2-0.999_EPOCH-500_BATCH-32_cc_v3.h5


2.7.0
Num GPUs Available:  1


2022-02-10 02:51:39.836794: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 02:51:39.876557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 02:51:39.876803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
dataset_root_dir = os.path.join("/home/charlesedwards/Documents", 'kaggle_2018_dataset')
data_npy_folder = os.path.join(dataset_root_dir, 'data')
model_dir = os.path.join( dataset_root_dir, 'models')

# we only care about test metadata
test_dir = os.path.join(dataset_root_dir, 'test')
test_catalog_csv = os.path.join(test_dir, 'catalog.csv')

test_metadata = pd.read_csv(test_catalog_csv)

# drop unwanted columns 
test_metadata.drop(['license','freesound_id'], axis=1, inplace=True)

In [3]:
# load in pre-processed MFCC train data
#X_train = np.load( os.path.join(data_npy_folder, "X-mfcc-train.npy" ) )
y_train = np.load( os.path.join(data_npy_folder, "y-mfcc-train.npy" ) )

# load in pre-processed MFCC train data
#X_test = np.load( os.path.join(data_npy_folder, "X-mfcc-test.npy" ) )
y_test = np.load( os.path.join(data_npy_folder, "y-mfcc-test.npy" ) )

In [4]:
model_dir = os.path.join( model_dir, 'MFCC_RESNET32_lr-1e-06_b1-0.99_b2-0.999_EPOCH-500_BATCH-32_cc_v3.h5')
model_dir

'/home/charlesedwards/Documents/kaggle_2018_dataset/models/MFCC_RESNET32_lr-1e-06_b1-0.99_b2-0.999_EPOCH-500_BATCH-32_cc_v3.h5'

In [5]:
model = load_model(model_dir)

2022-02-10 02:51:40.178144: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-10 02:51:40.179545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 02:51:40.179814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 02:51:40.180055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [92]:
n_mfcc = 40
sampling_rate = 44100
audio_duration = 2
audio_length = audio_duration * sampling_rate
input_shape = (n_mfcc, 1 + int(np.floor(audio_length/512)), 1)

def prepare(file_path:str):

    # using librosa to load the wav file
    y, sr = librosa.load(file_path,duration=audio_duration) # ,duration=5

    # normalize the audio with librosa
    y = librosa.util.normalize(y)

    # Extract MFCC data
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    # normalize the mfcc between -1 and 1
    #mfcc = librosa.util.normalize(mfcc)

    # reshape for neural network
    array = np.resize(mfcc, input_shape)
    return array.reshape(1, array.shape[0], array.shape[1], array.shape[2])

In [105]:
sample = test_metadata.sample()
wav_dir_of_sample = os.path.join(test_dir, sample['fname'].values[0])

# get prediction
predictions = model.predict([prepare(wav_dir_of_sample)])

index = np.argmax(predictions, axis=None, out=None)#np.argmax(predictions)
print(f"{sample['label'].values[0]} -> {y_test[index]}")

Saxophone -> Saxophone
